# Data Modeling and Partitioning in Azure Cosmos DB

## WebStore Demo

In [ ]:
# Setup

import json
import uuid

def printJson(result):
    print(json.dumps(result, indent=4))
    
def printCount(result):
    print("Retrieved " + str(len(result)) + " document(s)")
    
def printRequestCharge(container):
    print("Cost: " + container.client_connection.last_response_headers["x-ms-request-charge"] + " RU(s)")

In [ ]:
database                    = cosmos_client.get_database_client("webstore-v2")
customerContainer           = database.get_container_client("customer")
productCategoryContainer    = database.get_container_client("productCategory")
productContainer            = database.get_container_client("product")

### Querying for Customers

In [ ]:
# Retrieve a single customer (point read)

customer = customerContainer.read_item(
    "46192BCF-E8BB-4140-A0F1-B8764A7941E7",
    "46192BCF-E8BB-4140-A0F1-B8764A7941E7")

printJson(customer)
printRequestCharge(customerContainer)

In [ ]:
# Retrieve a single customer (SQL query)

customer = list(customerContainer.query_items(
    query = "SELECT * FROM c WHERE c.id = '46192BCF-E8BB-4140-A0F1-B8764A7941E7'"))[0]

printJson(customer)
printRequestCharge(customerContainer)

### Querying for Product Categories

In [ ]:
# Retrieve all product categories

productCategories = list(productCategoryContainer.query_items(
    query = "SELECT * FROM c WHERE c.type = 'category'"))

printJson(productCategories)
printCount(productCategories)
printRequestCharge(productCategoryContainer)

In [ ]:
# Project only the product category id and name properties

productCategories = list(productCategoryContainer.query_items(
    query = "SELECT c.id, c.name FROM c WHERE c.type = 'category'"))

printJson(productCategories)
printCount(productCategories)
printRequestCharge(productCategoryContainer)

### Using the Change Feed for Denormalization

In [ ]:
database                    = cosmos_client.get_database_client("webstore-v3")
productCategoryContainer    = database.get_container_client("productCategory")
productContainer            = database.get_container_client("product")

In [ ]:
# Retrieve the first 5 products from category 'Clothing, Shorts'

top5Products = list(productContainer.query_items(
    query = "SELECT TOP 5 * FROM c WHERE c.categoryId = 'C7324EF3-D951-45D9-A345-A82EAE344394'"))

printJson(top5Products)
printRequestCharge(customerContainer)

In [ ]:
# Rename product category to 'Clothing, Fun Shorts' (trigger change feed for Azure Function)

productCategory = productCategoryContainer.read_item("C7324EF3-D951-45D9-A345-A82EAE344394", "category")
printRequestCharge(productCategoryContainer)

productCategory["name"] = "Clothing, Fun Shorts"

productCategoryContainer.replace_item("C7324EF3-D951-45D9-A345-A82EAE344394", productCategory)
printRequestCharge(productCategoryContainer)

In [ ]:
# Retrieve the same products again (category name is updated by Azure Function)

top5Products = list(productContainer.query_items(
    query = "SELECT TOP 5 * FROM c WHERE c.categoryId = 'C7324EF3-D951-45D9-A345-A82EAE344394'"))

printJson(top5Products)
printRequestCharge(customerContainer)

### Querying for Sales Orders

In [ ]:
database            = cosmos_client.get_database_client("webstore-v4")
customerContainer   = database.get_container_client("customer")

In [ ]:
# Retrieve all sales order for a customer

orders = list(customerContainer.query_items(
    query = """
        SELECT *
        FROM c
        WHERE c.customerId = '46192BCF-E8BB-4140-A0F1-B8764A7941E7' AND c.type = 'salesOrder'
    """))

printJson(orders)
printCount(orders)
printRequestCharge(customerContainer)

In [ ]:
# Retrieve a customer with all their sales orders

customerWithOrders = list(customerContainer.query_items(
    query = """
        SELECT *
        FROM c
        WHERE c.customerId = '46192BCF-E8BB-4140-A0F1-B8764A7941E7'
        ORDER BY c.type
    """))

printJson(customerWithOrders)
printCount(customerWithOrders)
printRequestCharge(customerContainer)

### Querying for Top Customers

In [ ]:
# Retrieve the top 10 customers by number of sales orders

# Fails because we need to explicitly indicate that we want a cross-partition query
try:
    top10Customers = list(customerContainer.query_items(
        query = """
            SELECT TOP 10 c.id, c.firstName, c.lastName, c.salesOrderCount
            FROM c
            WHERE c.type = 'customer'
            ORDER BY c.salesOrderCount DESC
        """))
except Exception as e:
    print(e)

In [ ]:
# Try again with cross partition query enabled

top10Customers = list(customerContainer.query_items(
    query = """
        SELECT TOP 10 c.id, c.firstName, c.lastName, c.salesOrderCount
        FROM c
        WHERE c.type = 'customer'
        ORDER BY c.salesOrderCount DESC
    """,
    enable_cross_partition_query = True))

printJson(top10Customers)
printRequestCharge(customerContainer)

In [ ]:
# Create a new sales order for a customer

customerId = "44A6D5F6-AF44-4B34-8AB5-21C5DC50926E"
salesOrderId = str(uuid.uuid4())

newOrder = {
    "id": salesOrderId,
    "type": "salesOrder",
    "customerId": customerId,
    "details": [
        {
            "sku": "BK-R50R-44",
            "name": "Road-650 Red, 44",
            "price": 419.4589,
            "quantity": 1
        },
        {
            "sku": "BK-R68R-52",
            "name": "Road-450 Red, 52",
            "price": 874.794,
            "quantity": 1
        }
    ]
}

# Call the stored procedure to insert the new order and update the customer order count in a transaction
customerContainer.scripts.execute_stored_procedure(
    "spCreateSalesOrder",
    partition_key = customerId,
    params = newOrder)

printRequestCharge(customerContainer)

In [ ]:
# Top 10 query now shows updated result from salesOrderCount incremented by stored procedure

top10Customers = list(customerContainer.query_items(
    query = """
        SELECT TOP 10 c.id, c.firstName, c.lastName, c.salesOrderCount
        FROM c
        WHERE c.type = 'customer'
        ORDER BY c.salesOrderCount DESC
    """,
    enable_cross_partition_query = True))

printJson(top10Customers)
printRequestCharge(customerContainer)